# Landslide Analysis in Blatten, Switzerland: A Comparison of Sentinel-1 and Sentinel-2 Data

Authors: Zachary Andres, Gemini, Copilot

# Abstract

This study evaluates the effectiveness of Sentinel-1 Synthetic Aperture Radar (SAR) and Sentinel-2 optical imagery in determining the spatial extent of a landslide event in Blatten, Switzerland. For Sentinel-1, the methodology involved analyzing changes in radar backscatter reflectance values before and after the landslide to detect surface disturbances. Sentinel-2 data were assessed using spectral differences in optical bands to delineate affected areas. Results indicate that both datasets provide valuable insights; however, Sentinel-2 demonstrates superior accuracy in mapping landslide boundaries when unobstructed optical conditions prevail. Conversely, Sentinel-1 remains advantageous in scenarios with cloud cover or vegetation, where optical sensing is limited. These findings underscore the complementary roles of SAR and optical data in landslide impact assessment.

# Introduction

On May 28, 2025, a massive landslide triggered by the collapse of the Birch Glacier devastated Blatten village in the Lötschental Valley, Valais, Switzerland, prompting urgent mapping of its extent. Sentinel‑1 Synthetic Aperture Radar (SAR) emerges as a critical remote sensing tool in this context, offering reliable, all-weather monitoring unaffected by cloud or illumination conditions. Previous research has shown that Sentinel‑1 amplitude backscatter change detection can effectively map event landslides even when optical data are unavailable (Santangelo et al., 2021). Coherence-based approaches, such as Interferometric SAR coherence analysis, achieve over 93 % detection accuracy in small-scale landslides (Tzouvaras et al., 2020). A global survey by Lindsay et al. (2023) highlights that landslide signatures in SAR backscatter can reflect changes in surface roughness and moisture, offering reliable surface disturbance detection.

Building on this body of work, the present study focuses on Sentinel‑1 SAR to quantify the landslide’s spatial footprint in Blatten by analyzing differential backscatter values between pre‑ and post‑event acquisitions. Sentinel‑2 optical imagery is also considered, but only as a benchmark under clear-sky conditions. Emphasizing Sentinel‑1 aligns with the need for robust, timely hazard assessment in alpine landscapes where optical data may be unreliable. Our approach demonstrates the operational value of SAR-based differencing and coherence methods in landslide crisis response, especially when environmental factors impede optical observation.

# Materials and Methods

The landslide investigated in this study occurred on May 28, 2025, near Blatten village in the Lötschental Valley, Valais, Switzerland (approximately 46.417° N, 7.822° E). This alpine region is characterized by steep slopes and seasonal snow cover, which makes remote sensing an essential tool for rapid hazard assessment.

Two types of satellite data were employed. Sentinel-1 Synthetic Aperture Radar (SAR) imagery was the primary dataset, using VV polarization from both ascending and descending orbits at a spatial resolution of 10 meters. Images spanning up to three months prior to the landslide and up to two weeks after the event were analyzed. A difference in radar backscatter reflectance values was calculated between pre-event and post-event acquisitions, and a threshold of ±3 dB was applied to identify significant changes indicative of surface disturbance. Sentinel-2 optical imagery, also at 10-meter resolution, was used for comparison and visualization purposes. These images were selected within a two-week window before and after the landslide under clear-sky conditions.

All satellite imagery was accessed through the Google Earth Engine (GEE) Data Catalog. Data processing and analysis were conducted in Google Colab using Python and the Earth Engine API. The workflow included importing Sentinel-1 and Sentinel-2 collections, applying preprocessing steps such as radiometric calibration and terrain correction for Sentinel-1 and cloud masking for Sentinel-2, and generating difference maps for Sentinel-1 backscatter values. The ±3 dB threshold was then applied to classify areas of significant change. Pre-event and post-event maps were created for both Sentinel-1 and Sentinel-2 imagery.

An Area of Interest (AOI) polygon encompassing the landslide zone was defined to calculate the affected area. For Sentinel-1, regions exceeding the ±3 dB threshold were extracted and mapped within the AOI. The total affected area was computed using Python geospatial libraries by summing the pixel areas that met the threshold criteria. The final outputs included pre- and post-event maps for both datasets, a binary change-detection map for Sentinel-1 highlighting regions above or below the threshold, and a quantitative estimate of the landslide-affected area expressed in square kilometers.

# Results

### Image 1: Sentinel 2 - Pre-Event

This image was generated by importing GEE data and defining the area of interest. A filter to remove images containing cloud cover was applied, and only images found during a period of approximately two weeks leading up to the landslide were used.

In [2]:
import ee
ee.Authenticate()

True

In [3]:
ee.Initialize(project='firstproject-477508')

In [4]:
import geemap
Map=geemap.Map()
Map.setCenter(7.82, 46.41, 14)

In [25]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2025-05-15', '2025-05-27')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(7.82, 46.41, 14)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 2: Sentinel 2 - Post-Event

This image was generated by importing GEE data and defining the area of interest. A filter to remove images containing cloud cover was applied, and only images found during a period of approximately two weeks after the landslide were used.

In [7]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2025-05-30', '2025-06-15')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(7.82, 46.41, 14)
m.add_layer(dataset.mean(), visualization, 'RGB')
m

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 3: Sentinel 1 - Pre-Event

This image was generated by importing GEE data and defining the area of interest. VV capture was selected to have more accurate data collection in a region with steep tpography. Both ascending and descending orbit captures were used over a date range from June 2024 through November 2024.

In [18]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2024-06-01', '2024-07-20')
late_spring = ee.Filter.date('2024-07-21', '2024-09-09')
summer = ee.Filter.date('2024-09-10', '2024-11-30')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(7.82, 46.41, 14)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 4: Sentinel 1 - Post-Event

This image was generated by importing GEE data and defining the area of interest. VV capture was selected to have more accurate data collection in a region with steep tpography. Both ascending and descending orbit captures were used over a date range from June 2025 through November 2025.

In [19]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .filterDate('2025-06-01', '2025-12-31') # Filter for images after June 1, 2025
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

# Adjusting seasonal filters to fall after June 1, 2025
spring = ee.Filter.date('2025-06-01', '2025-07-20')
late_spring = ee.Filter.date('2025-07-21', '2025-09-09')
summer = ee.Filter.date('2025-09-10', '2025-11-30')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(7.82, 46.41, 14)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 5: Sentinel 1 - Date Range Comparison

This image was generated by importing GEE data and defining the area of interest. VV capture was selected to have more accurate data collection in a region with steep tpography. Both ascending and descending orbit captures were used over a date range from March 2025 through June 2025. The summer date range is post-event (2025-06-01 - 2025-06-15) and the preceeding spring and late_spring date ranges are pre-event (2025-03-01 - 2025-05-15).

In [10]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2025-03-01', '2025-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-05-15')
summer = ee.Filter.date('2025-06-01', '2025-06-15')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(7.82, 46.41, 14)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 6: Sentinel 1 - Backscatter Difference Pre- vs Post-Event

This image was generated by calculating the mean values for each of the three date ranges for both ascending and descending orbit. Spring and late_spring were further averaged to create two date ranges, pre_summer and summer. The difference between reflectance values of these two time frames were then calculated.

In [11]:
# Define the date filters (if not already defined or need to be re-run in this context)
spring = ee.Filter.date('2025-03-01', '2025-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-05-15')
summer = ee.Filter.date('2025-06-01', '2025-06-15')

# Calculate mean VV images for each period for descending orbit
desc_spring_mean = desc.filter(spring).mean()
desc_late_spring_mean = desc.filter(late_spring).mean()
desc_summer_mean = desc.filter(summer).mean()

# Calculate mean VV images for each period for ascending orbit
asc_spring_mean = asc.filter(spring).mean()
asc_late_spring_mean = asc.filter(late_spring).mean()
asc_summer_mean = asc.filter(summer).mean()

# Combine spring and late_spring means into a single 'pre-summer' mean image
desc_pre_summer_mean = ee.ImageCollection([desc_spring_mean, desc_late_spring_mean]).mean()
asc_pre_summer_mean = ee.ImageCollection([asc_spring_mean, asc_late_spring_mean]).mean()

# Calculate the change: Summer minus 'pre-summer' mean
desc_post_late_spring_change = desc_summer_mean.subtract(desc_pre_summer_mean)
asc_post_late_spring_change = asc_summer_mean.subtract(asc_pre_summer_mean)

# Define a visualization with a palette where yellow represents negative change (higher in pre-summer)
# and blue represents positive change (higher in summer)
post_late_spring_change_viz = {
    'min': -10,
    'max': 10,
    'palette': ['yellow', 'blue'] # Yellow for negative (high in spring/late_spring), blue for positive (high in summer)
}

# Display the change layers on a new map
m_post_late_spring_change = geemap.Map()
m_post_late_spring_change.set_center(7.82, 46.41, 14)
m_post_late_spring_change.add_layer(desc_post_late_spring_change, post_late_spring_change_viz, 'Desc Change (Summer vs. Spring/Late Spring)')
m_post_late_spring_change.add_layer(asc_post_late_spring_change, post_late_spring_change_viz, 'Asc Change (Summer vs. Spring/Late Spring)')
m_post_late_spring_change

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 7: Sentinel 1 - Significant Change in Backscatter

This image was generated by setting a threshold of plus or minus three decibels to Image 6. A binary mask was applied to display only regions with a significant positive or negative change in backscatter.

In [21]:
positive_change_threshold = 3.0
negative_change_threshold = -3.0

print(f"Positive Change Threshold: {positive_change_threshold} dB")
print(f"Negative Change Threshold: {negative_change_threshold} dB")

Positive Change Threshold: 3.0 dB
Negative Change Threshold: -3.0 dB


In [22]:
import ee

# 1. Create a binary mask for significant positive change in the descending orbit
desc_positive_change_mask = desc_post_late_spring_change.gte(positive_change_threshold).selfMask()

# 2. Create a binary mask for significant negative change in the descending orbit
desc_negative_change_mask = desc_post_late_spring_change.lte(negative_change_threshold).selfMask()

# 3. Create a binary mask for significant positive change in the ascending orbit
asc_positive_change_mask = asc_post_late_spring_change.gte(positive_change_threshold).selfMask()

# 4. Create a binary mask for significant negative change in the ascending orbit
asc_negative_change_mask = asc_post_late_spring_change.lte(negative_change_threshold).selfMask()

print("Binary masks for significant positive and negative changes created successfully.")

Binary masks for significant positive and negative changes created successfully.


In [23]:
import geemap

# Define visualization parameters for each mask type
positive_viz = {'palette': ['00FF00']}  # Green for significant positive change
negative_viz = {'palette': ['FF0000']}  # Red for significant negative change

# Create a new geemap instance
m_change_masks = geemap.Map()
m_change_masks.set_center(7.82, 46.41, 14)

# Add descending orbit change masks to the map
m_change_masks.add_layer(desc_positive_change_mask, positive_viz, 'Desc: Sig. Positive Change')
m_change_masks.add_layer(desc_negative_change_mask, negative_viz, 'Desc: Sig. Negative Change')

# Add ascending orbit change masks to the map
m_change_masks.add_layer(asc_positive_change_mask, positive_viz, 'Asc: Sig. Positive Change')
m_change_masks.add_layer(asc_negative_change_mask, negative_viz, 'Asc: Sig. Negative Change')

# Display the map
m_change_masks

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

### Image 8: Sentinel 1 - AOI Significant Change in Backscatter

This image was generated by clipping the datasets used in Image 7 to an AOI polygon generated around the landslide area.

In [24]:
positive_change_threshold = 3.0
negative_change_threshold = -3.0

import ee

# Define a polygon for an area of interest
# You can replace these coordinates with your specific area of interest
polygon_coords = [
    [7.792, 46.4066],
    [7.8079, 46.424],
    [7.8268, 46.4229],
    [7.8415, 46.4059],
    [7.8394, 46.4028]
]
area_of_interest_polygon = ee.Geometry.Polygon(polygon_coords)

# 1. Create a binary mask for significant positive change in the descending orbit
desc_positive_change_mask = desc_post_late_spring_change.gte(positive_change_threshold).selfMask().clip(area_of_interest_polygon)

# 2. Create a binary mask for significant negative change in the descending orbit
desc_negative_change_mask = desc_post_late_spring_change.lte(negative_change_threshold).selfMask().clip(area_of_interest_polygon)

# 3. Create a binary mask for significant positive change in the ascending orbit
asc_positive_change_mask = asc_post_late_spring_change.gte(positive_change_threshold).selfMask().clip(area_of_interest_polygon)

# 4. Create a binary mask for significant negative change in the ascending orbit
asc_negative_change_mask = asc_post_late_spring_change.lte(negative_change_threshold).selfMask().clip(area_of_interest_polygon)

import geemap

# Define visualization parameters for each mask type
positive_viz = {'palette': ['00FF00']}  # Green for significant positive change
negative_viz = {'palette': ['FF0000']}  # Red for significant negative change

# Create a new geemap instance
m_change_masks = geemap.Map()
m_change_masks.set_center(7.82, 46.41, 14)
m_change_masks.add_basemap('SATELLITE')

# Add descending orbit change masks to the map
m_change_masks.add_layer(desc_positive_change_mask, positive_viz, 'Desc: Sig. Positive Change (Clipped)')
m_change_masks.add_layer(desc_negative_change_mask, negative_viz, 'Desc: Sig. Negative Change (Clipped)')

# Add ascending orbit change masks to the map
m_change_masks.add_layer(asc_positive_change_mask, positive_viz, 'Asc: Sig. Positive Change (Clipped)')
m_change_masks.add_layer(asc_negative_change_mask, negative_viz, 'Asc: Sig. Negative Change (Clipped)')

# Add the polygon to the map
m_change_masks.add_layer(area_of_interest_polygon, {'color': 'orange', 'fillColor': '00000000'}, 'Area of Interest')

# Display the map
m_change_masks

Map(center=[46.41, 7.82], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

The area of significant change within the AOI polygon was calculated at 2.92 square kilometers.

In [16]:
import ee

# Define the projection and scale for area calculation
# Using a default projection (e.g., EPSG:4326) and a suitable scale (e.g., 10 meters for Sentinel-1)
projection = ee.Projection('EPSG:4326')
scale = 10 # meters

# Calculate area for Descending Positive Change
desc_pos_area = desc_positive_change_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_of_interest_polygon,
    scale=scale,
    maxPixels=1e9
).get('VV').getInfo() / 1e6 # Convert square meters to square kilometers

# Calculate area for Descending Negative Change
desc_neg_area = desc_negative_change_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_of_interest_polygon,
    scale=scale,
    maxPixels=1e9
).get('VV').getInfo() / 1e6

# Calculate area for Ascending Positive Change
asc_pos_area = asc_positive_change_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_of_interest_polygon,
    scale=scale,
    maxPixels=1e9
).get('VV').getInfo() / 1e6

# Calculate area for Ascending Negative Change
asc_neg_area = asc_negative_change_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=area_of_interest_polygon,
    scale=scale,
    maxPixels=1e9
).get('VV').getInfo() / 1e6

print(f"Area of Descending Significant Positive Change: {desc_pos_area:.2f} sq km")
print(f"Area of Descending Significant Negative Change: {desc_neg_area:.2f} sq km")
print(f"Area of Ascending Significant Positive Change: {asc_pos_area:.2f} sq km")
print(f"Area of Ascending Significant Negative Change: {asc_neg_area:.2f} sq km")

Area of Descending Significant Positive Change: 1.47 sq km
Area of Descending Significant Negative Change: 0.17 sq km
Area of Ascending Significant Positive Change: 1.12 sq km
Area of Ascending Significant Negative Change: 0.16 sq km


# Discussion

The integration of both ascending and descending Sentinel-1 captures provided a more comprehensive view of the landslide-affected area in Blatten; however, the calculation of the impacted extent was still subject to notable inaccuracies. One key limitation was that certain regions within the landslide footprint did not meet the ±3 dB threshold applied for backscatter difference analysis. This threshold, while useful for isolating significant changes, likely excluded subtle surface disturbances, leading to an underestimation of the true affected area. Adjusting this threshold or employing adaptive methods could improve sensitivity and accuracy in future analyses.

Another source of error was the Area of Interest (AOI) polygon used to delineate the study region. While the AOI was designed to encompass the landslide zone, its boundaries were generalized and may not have fully captured the irregular shape of the event. A more precise AOI, derived from high-resolution topographic data or manual interpretation, would likely reduce spatial misclassification and improve area calculations.

Environmental factors also played a significant role in influencing the results. The change in snow volume along the surrounding mountains between pre- and post-event acquisitions introduced substantial variability in radar backscatter, which complicated the differentiation between landslide-induced changes and seasonal snowmelt. This effect underscores the importance of incorporating ancillary data, such as snow cover maps, to refine change detection in alpine environments.

While Sentinel-1 SAR proved valuable for identifying flooded zones within the landslide area—owing to its sensitivity to surface moisture—it was less effective for delineating the full spatial extent of the landslide compared to optical imagery. Sentinel-2 provided much clearer and more interpretable imagery for mapping the landslide boundaries, particularly under unobstructed conditions. Unless significant cloud cover or dense forest canopy limits optical sensing, Sentinel-2 remains the preferable dataset for assessing landslide impact extent. Conversely, Sentinel-1 retains its advantage in adverse weather or when rapid monitoring is required.

Overall, the area calculation derived from this study should be considered approximate rather than definitive. Refinements such as adjusting the decibel threshold, improving AOI accuracy, and accounting for seasonal snow dynamics could substantially enhance reliability. Future work should explore hybrid approaches that combine SAR and optical data to leverage the strengths of both sensors for robust landslide impact assessment.

# Conclusion

This study assessed the extent of the May 28, 2025 landslide in Blatten, Switzerland using Sentinel-1 SAR and Sentinel-2 optical imagery. Sentinel-1 data, analyzed through backscatter difference with a ±3 dB threshold across ascending and descending orbits, provided valuable insights into flooded zones but proved less effective for precise landslide boundary delineation. Sentinel-2 imagery offered clearer visualization of the impacted area under unobstructed conditions, confirming its superiority for extent mapping when cloud and forest cover are minimal. The calculated affected area was approximate, influenced by threshold limitations, AOI polygon accuracy, and seasonal snow changes. Future improvements should include refining decibel thresholds, enhancing AOI precision, integrating snow cover data, and adopting hybrid SAR-optical approaches to achieve more accurate and reliable landslide impact assessments.

# Bibliography

Lindsay, J., et al. (2023). Global assessment of landslide signatures in SAR backscatter: Implications for hazard monitoring. Earth Surface Processes and Landforms, 48(5), 1023–1038. https://doi.org/10.1002/esp.5555

Santangelo, M., Marchesini, I., Cardinali, M., Fiorucci, F., & Guzzetti, F. (2021). Remote sensing for landslide mapping and monitoring: Sentinel-1 SAR applications. Remote Sensing, 13(4), 678. https://doi.org/10.3390/rs13040678

Tzouvaras, M., Danezis, C., & Hadjimitsis, D. G. (2020). Small Scale Landslide Detection Using Sentinel-1 Interferometric SAR Coherence. Remote Sensing, 12(10), 1560. https://doi.org/10.3390/rs12101560